<a href="https://colab.research.google.com/github/JAMES-YI/00_Python_Tutorial/blob/master/Lab_01_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab_01 of ECE 5995: Foundations of Deep Learning
This file is to demonstrate basic ideas of convolutional neural networks, and the following techniques are covered:

(1) Construct a CNN

(2) Train a CNN

(3) Evaluation of a trained CNN

(4) Visualization of CNN kernels

(5) Visualization of learned representations after each layer


Code from: https://www.tensorflow.org/tutorials/images/cnn

Modified by JYI, 10/28/2019


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

from keras import datasets, layers, models, optimizers
import matplotlib.pyplot as plt
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


In [0]:
# Load data
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

class_names = ['0', '1', '2', '3', '4',
               '5', '6', '7', '8', '9']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i]])
plt.show()

In [0]:
# CNN with 3 convolutional layers and 2 fully connected layers

KN1 = 16
KN2 = 16
KN3 = 16

model = models.Sequential()
model.add(layers.Conv2D(16, (3, 3), strides=(1, 1), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(16, (3, 3), strides=(1, 1), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(16, (3, 3), strides=(1, 1), activation='relu'))

model.add(layers.Flatten())

model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

In [0]:
# CNN training

sgd = optimizers.SGD(lr=0.01)
model.compile(optimizer=sgd,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_images.reshape([-1,28,28,1]), train_labels, epochs=5, 
                    validation_data=(test_images.reshape([-1,28,28,1]), test_labels))

In [0]:
# Performance evaluation

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Acc')
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images.reshape([-1,28,28,1]),  test_labels, verbose=2)
print(f"Test loss: {test_loss}, Test accuracy: {test_acc}")

In [0]:
# What CNN learned: weight patterns
import numpy as np

Conv1W = model.layers[2].get_weights()[0]
plt.figure(figsize=(12,12))
for i in np.arange(KN1):
  plt.subplot(4,int(KN1/4),i+1)
  plt.imshow(Conv1W[:,:,0,i])
  plt.title('Kernel' + str(i+1))
  plt.colorbar()

plt.subplots_adjust(top = 0.99, bottom=0.01, hspace=1.5, wspace=0.4)
plt.show()


In [0]:
# What CNN learned: new representation after each layer
from keras import backend as K
import numpy as np

IP = model.input
OPs = [layer.output for layer in model.layers]  
funcs = [K.function([IP, K.learning_phase()], [out]) for out in OPs]

# Random input and corresponding output after each layer
IPVal1 = np.random.random([1,28,28,1])
OPVal1 = [func([IPVal1, 1.]) for func in funcs]

# Natural input and corresponding output after each layer 
IPVal2 = train_images[0].reshape([1,28,28,1])
OPVal2 = [func([IPVal2, 1.]) for func in funcs]

In [0]:
plt.figure()
plt.subplot(1,2,1)
plt.imshow(IPVal2.reshape([28,28]))
plt.title('Natural:' + str(train_labels[0]))
plt.subplot(1,2,2)
plt.imshow(IPVal1.reshape([28,28]))
plt.title('Random')
plt.show()

In [0]:
# What CNN learned: new representation after each layer (II)
Conv1OP1 = OPVal1[0][0] # rand
Conv1OP2 = OPVal2[0][0] # nat

plt.figure(figsize=(12,12))
for i in np.arange(KN1):
  plt.subplot(4,int(KN1/4),i+1)
  plt.imshow(Conv1OP1[0,:,:,i])
  plt.title('Rand: c ' + str(i+1))
  plt.colorbar()

plt.subplots_adjust(top = 0.99, bottom=0.01, hspace=0.5, wspace=0.4)
plt.show()

plt.figure(figsize=(12,12))
for i in np.arange(KN1):
  plt.subplot(4,int(KN1/4),i+1)
  plt.imshow(Conv1OP2[0,:,:,i])
  plt.title('Natl: c ' + str(i))
  plt.colorbar()

plt.subplots_adjust(top = 0.99, bottom=0.01, hspace=0.5, wspace=0.4)
plt.show()